In [1]:
import os
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
def load_from_gdrive(data_dir):
  # Initialize an empty list to store the DataFrames
  data_frames = []

  # Loop through each .csv file in the directory
  for dir in data_dir:
    for file_name in os.listdir(dir):
        if file_name.endswith('.csv'):
            file_path = os.path.join(dir, file_name)

            # Load the .csv file into a DataFrame
            df = pd.read_csv(file_path)

            # Remove the first column
            df = df.iloc[:, 4:]

            df  = df.div(360)

            df  = df.clip(upper =1,lower = -1)


            # Append the DataFrame to the list
            data_frames.append(df)

  # Concatenate the DataFrames into a single DataFrame
  combined_df = pd.concat(data_frames, axis=0, ignore_index=True)
  return combined_df

In [3]:
data_dir = ['/content/drive/MyDrive/28 08 mod/Normal']

combined_df =  load_from_gdrive(data_dir)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F

class ComplexVAEFFT(nn.Module):
    def __init__(self, input_size, latent_size, neurons, dropout_rate=0.5, weight_decay=0.01):
        super(ComplexVAEFFT, self).__init__()
        self.latent_size = latent_size

        self.encoder = nn.Sequential(
              nn.Linear(input_size * 2, neurons),
              nn.ReLU(),
              nn.Dropout(dropout_rate),
              nn.Linear(neurons, neurons // 2),
              nn.ReLU(),
              nn.Dropout(dropout_rate),
              nn.Linear(neurons // 2, neurons // 4),
              nn.ReLU(),
              nn.Linear(neurons // 4, self.latent_size * 2)
          )

        self.decoder = nn.Sequential(
              nn.Linear(self.latent_size, neurons // 4),
              nn.ReLU(),
              nn.Dropout(dropout_rate),
              nn.Linear(neurons // 4, neurons // 2),
              nn.ReLU(),
              nn.Dropout(dropout_rate),
              nn.Linear(neurons // 2, neurons),
              nn.ReLU(),
              nn.Linear(neurons, input_size * 2)
          )

        self.weight_decay = weight_decay

    def forward(self, x):
        # Apply FFT to input
        x_fft = torch.fft.fft(x)

        # Separate real and imaginary parts
        x_fft_real = x_fft.real
        x_fft_imag = x_fft.imag
        x_fft_separated = torch.cat((x_fft_real, x_fft_imag), dim=-1)

        latent_params = self.encoder(x_fft_separated)
        mu = latent_params[:, :self.latent_size]  # Use self.latent_size
        logvar = latent_params[:, self.latent_size:]  # Use self.latent_size
        z = self.reparameterize(mu, logvar)
        reconstruction_separated = self.decoder(z)

        # Combine real and imaginary parts
        reconstruction_real = reconstruction_separated[:, :input_size]
        reconstruction_imag = reconstruction_separated[:, input_size:]
        reconstruction = torch.complex(reconstruction_real, reconstruction_imag)

        # Apply inverse FFT to output
        reconstruction_ifft = torch.fft.ifft(reconstruction)

        return reconstruction_ifft, mu, logvar


    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std


def complex_mse_loss(reconstruction, x):
    # Compute MSE for real part
    reconstruction_real = reconstruction.real
    real_loss = nn.functional.mse_loss(reconstruction_real, x, reduction='sum')

    # Compute MSE for imaginary part
    if torch.is_complex(x):
        x_imag = x.imag
        reconstruction_imag = reconstruction.imag
        imag_loss = nn.functional.mse_loss(reconstruction_imag, x_imag, reduction='sum')
    else:
        imag_loss = 0

    return real_loss + imag_loss

def vae_loss(reconstruction, x, mu, logvar):
    recon_loss = complex_mse_loss(reconstruction, x)
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Add weight decay loss
    weight_decay_loss = 0
    for param in vae.parameters():
        weight_decay_loss += torch.sum(torch.square(param))

    return recon_loss + kl_divergence + (weight_decay_loss * vae.weight_decay)



def train_vae(model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        reconstruction, mu, logvar = model(batch)
        loss = criterion(reconstruction, batch, mu, logvar)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    return train_loss / len(train_loader)

def test_vae(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            reconstruction, mu, logvar = model(batch)
            loss = criterion(reconstruction, batch, mu, logvar)
            test_loss += loss.item()
    return test_loss / len(test_loader)


# Parameters
input_size = 59
epochs = 50
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#( input_size, latent_size, neurons, dropout_rate=0.5, weight_decay=0.01)
vae = ComplexVAEFFT(input_size, 64,1024,0,0).to(device)

optimizer = torch.optim.Adam(vae.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

tensor_data = torch.tensor(combined_df.values, dtype=torch.float32)
train_data, test_data = train_test_split(tensor_data, test_size=0.4,random_state = 42)
validation_data, test_data = train_test_split(test_data, test_size=0.5,random_state = 42)


for epoch in range(epochs):

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    train_loss = train_vae(vae, train_loader, optimizer, vae_loss, device)
    test_loss = test_vae(vae, test_loader, vae_loss, device)

    print(f'Epoch {epoch+1}/{epochs},  Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F

class ComplexVAEFFT(nn.Module):
    def __init__(self, input_size, latent_size, neurons, dropout_rate=0.5, weight_decay=0.01):
        super(ComplexVAEFFT, self).__init__()
        self.latent_size = latent_size



        self.encoder = nn.Sequential(
              nn.Linear(input_size * 2, neurons),
              nn.ReLU(),
              nn.Dropout(dropout_rate),
              nn.Linear(neurons, neurons // 2),
              nn.ReLU(),
              nn.Dropout(dropout_rate),
              nn.Linear(neurons // 2, neurons // 4),
              nn.ReLU(),
              nn.Linear(neurons // 4, self.latent_size * 2)
          )

        self.decoder = nn.Sequential(
              nn.Linear(self.latent_size, neurons // 4),
              nn.ReLU(),
              nn.Dropout(dropout_rate),
              nn.Linear(neurons // 4, neurons // 2),
              nn.ReLU(),
              nn.Dropout(dropout_rate),
              nn.Linear(neurons // 2, neurons),
              nn.ReLU(),
              nn.Linear(neurons, input_size * 2)
          )

        self.weight_decay = weight_decay

    def forward(self, x):
        # Apply FFT to input
        x_fft = torch.fft.fft(x)

        # Separate real and imaginary parts
        x_fft_real = x_fft.real
        x_fft_imag = x_fft.imag
        x_fft_separated = torch.cat((x_fft_real, x_fft_imag), dim=-1)

        latent_params = self.encoder(x_fft_separated)
        mu = latent_params[:, :self.latent_size]  # Use self.latent_size
        logvar = latent_params[:, self.latent_size:]  # Use self.latent_size
        z = self.reparameterize(mu, logvar)
        reconstruction_separated = self.decoder(z)

        # Combine real and imaginary parts
        reconstruction_real = reconstruction_separated[:, :input_size]
        reconstruction_imag = reconstruction_separated[:, input_size:]
        reconstruction = torch.complex(reconstruction_real, reconstruction_imag)

        # Apply inverse FFT to output
        reconstruction_ifft = torch.fft.ifft(reconstruction)

        return reconstruction_ifft, mu, logvar


    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std


def complex_mse_loss(reconstruction, x):
    # Compute MSE for real part
    reconstruction_real = reconstruction.real
    real_loss = nn.functional.mse_loss(reconstruction_real, x, reduction='sum')

    # Compute MSE for imaginary part
    if torch.is_complex(x):
        x_imag = x.imag
        reconstruction_imag = reconstruction.imag
        imag_loss = nn.functional.mse_loss(reconstruction_imag, x_imag, reduction='sum')
    else:
        imag_loss = 0

    return real_loss + imag_loss


def vae_loss(reconstruction, x, mu, logvar):
    recon_loss = complex_mse_loss(reconstruction, x)
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Add weight decay loss
    weight_decay_loss = 0
    for param in vae.parameters():
        weight_decay_loss += torch.sum(torch.square(param))

    return recon_loss + kl_divergence + (weight_decay_loss * vae.weight_decay)



def train_vae(model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        reconstruction, mu, logvar = model(batch)
        loss = criterion(reconstruction, batch, mu, logvar)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    return train_loss / len(train_loader)

def test_vae(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            reconstruction, mu, logvar = model(batch)
            loss = criterion(reconstruction, batch, mu, logvar)
            test_loss += loss.item()
    return test_loss / len(test_loader)

from google.colab import drive
import pandas as pd
from sklearn.model_selection import ParameterGrid
import torch
import os

# Mount your Google Drive
drive.mount('/content/gdrive')

# Create directories in your Google Drive to store the models and logs
os.makedirs('/content/gdrive/My Drive/VAE_FFT_5layer', exist_ok=True)
os.makedirs('/content/gdrive/My Drive/VAE_FFT_5layer', exist_ok=True)

# Define the grid of parameters
param_grid = [{'lr': [0.0001, 0.00001],
               'weight_decay': [0],
               'dropout': [0,0.2,0.4],
               'batch_size': [32, 64],
               'latent_size': [64, 128],
               'neurons': [512,1024]}]


grid = ParameterGrid(param_grid)

# Early stopping parameters
input_size = 59
epochs = 50
n_epochs_stop = 5
epochs_no_improve = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tensor_data = torch.tensor(combined_df.values, dtype=torch.float32)
train_data, test_data = train_test_split(tensor_data, test_size=0.4,random_state = 42)
validation_data, test_data = train_test_split(test_data, test_size=0.5,random_state = 42)

# Grid search
for params in grid:
    best_test_loss = float('inf')
    # Instantiate
    vae = ComplexVAEFFT(input_size, params['latent_size'],params['neurons'],params['dropout'],params['weight_decay']).to(device)
    optimizer = torch.optim.Adam(vae.parameters(), lr=params['lr'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # DataLoaders
    train_loader = DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
    test_loader = DataLoader(test_data, batch_size=params['batch_size'], shuffle=False)

    print(f'with parameters: {params}')

    # Train VAE
    for epoch in range(epochs):

        train_loss = train_vae(vae, train_loader, optimizer, vae_loss, device)
        test_loss = test_vae(vae, test_loader, vae_loss, device)

        print(f'Epoch {epoch+1}/{epochs},  Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

        # Early stopping
        if test_loss < best_test_loss:
            epochs_no_improve = 0
            best_test_loss = test_loss
        else:
            epochs_no_improve += 1
        if epochs_no_improve == n_epochs_stop:
            print('Early stopping!')
            break

        scheduler.step()

    # Define the paths for saving models and logs
    model_path = model_path = f'/content/gdrive/My Drive/VAE_FFT_5layer/{test_loss:.4f}_lr={params["lr"]}_batch={params["batch_size"]}_latent={params["latent_size"]}_neurons={params["neurons"]}_dropout={params["dropout"]}_reg={params["weight_decay"]}.pt'
    log_path = f'/content/gdrive/My Drive/VAE_FFT_5layer/{test_loss:.4f}_lr={params["lr"]}_batch={params["batch_size"]}_latent={params["latent_size"]}_neurons={params["neurons"]}_dropout={params["dropout"]}_reg={params["weight_decay"]}_log.csv'

    # Save the model
    torch.save(vae.state_dict(), model_path)
    print(f'Model saved to: {model_path}')

    # Save the log
    log_df = pd.DataFrame({'parameters': str(params),
                           'train_loss': [train_loss],
                           'test_loss': [test_loss]})
    log_df.to_csv(log_path, index=False)
    print(f'Log saved to: {log_path}')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
with parameters: {'batch_size': 32, 'dropout': 0, 'latent_size': 64, 'lr': 0.0001, 'neurons': 512, 'weight_decay': 0}
Epoch 1/50,  Train Loss: 10.9670, Test Loss: 10.7606
Epoch 2/50,  Train Loss: 10.5743, Test Loss: 10.7377
Epoch 3/50,  Train Loss: 10.5585, Test Loss: 10.7372
Epoch 4/50,  Train Loss: 10.5475, Test Loss: 10.7261
Epoch 5/50,  Train Loss: 10.5403, Test Loss: 10.7063
Epoch 6/50,  Train Loss: 10.5353, Test Loss: 10.7057
Epoch 7/50,  Train Loss: 10.5304, Test Loss: 10.7063
Epoch 8/50,  Train Loss: 10.5270, Test Loss: 10.7038
Epoch 9/50,  Train Loss: 10.5270, Test Loss: 10.6953
Epoch 10/50,  Train Loss: 10.5253, Test Loss: 10.7017


KeyboardInterrupt: ignored